# Identificación de tumores de mama a partir del análisis de imágenes con técnicas de aprendizaje profundo

## CNNs

## Preprocesamiento del dataset

### Importación de librerías

In [ ]:
#CNNs
import os
import pydicom
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.regularizers import L1
from tensorflow.keras.regularizers import L2
from tensorflow.keras.regularizers import L1L2

# YOLO
import shutil
from ultralytics import YOLO
from collections import defaultdict

### Clasificación y transformación de las imágenes

In [ ]:
# Se determinan las rutas de los archivos
base_dir = r'C:\Users\mcamp\OneDrive\Documentos\TFGDEF2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Training'
output_dir = Path(base_dir)

# Se definen las carpetas en las que serán clasificadas las mamografías
main_folders = ["ROI mask images", "full mammogram images", "cropped images"]

def create_main_folders():
    """
    Función destinada a la creación de carpetas principales si no existen previamente
    """
    for folder in main_folders:
        path = output_dir / folder
        if not path.exists():
            path.mkdir(parents=True)

def resize_with_padding(image, target_size, interpolation=cv2.INTER_AREA):
    """
    Función destinada a la redimensión de las imágenes añadiendo bordes negros
    """
    old_size = image.shape[:2]
    ratio = min(target_size[1] / old_size[0], target_size[0] / old_size[1])
    new_size = (int(old_size[1] * ratio), int(old_size[0] * ratio))

    resized_image = cv2.resize(image, new_size, interpolation=interpolation)

    delta_w = target_size[0] - new_size[0]
    delta_h = target_size[1] - new_size[1]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    # Agrega los bordes
    color = [0, 0, 0] # negro
    padded_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return padded_image

def preprocess_image(source_path, target_path, target_size):
    """
    Función destinada al procesamiento y guardado de imágenes.
    Lee un archivo DICOM, lo redimensiona añadiendo un borde negro y lo guarda como .png.
    """
    try:
        # Extrae los datos de píxeles
        dicom_data = pydicom.dcmread(source_path)
        image_array = dicom_data.pixel_array

        # Normaliza la imagen a 8 bits (0-255) si es necesario
        if image_array.max() > 255:
            image_array = (image_array / image_array.max() * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)

        # Redimensiona la imagen añadiendo borde negro
        resized_image = resize_with_padding(image_array, target_size, interpolation=cv2.INTER_AREA)

        # Guarda como imagen procesada
        cv2.imwrite(target_path, resized_image)
        print(f"Procesada y guardada en: {target_path}")
    except Exception as e:
        print(f"Error al procesar {source_path}: {e}")

def reorganize_images():
    """
    Función principal destinada a la reorganización y transformación de imágenes.
    """
    create_main_folders()

    # Viaja entre las carpetas del dataset hasta alcanzar los documentos fundamentales para el estudio
    for class_folder in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_folder)
        if os.path.isdir(class_path):
            print(f"Clase: {class_folder}")

            for first_level_folder in os.listdir(class_path):
                first_level_path = os.path.join(class_path, first_level_folder)
                if os.path.isdir(first_level_path):
                    print(f"Primer nivel: {first_level_folder}")

                    for second_level_folder in os.listdir(first_level_path):
                        second_level_path = os.path.join(first_level_path, second_level_folder)
                        if os.path.isdir(second_level_path):
                            print(f"Segundo nivel: {second_level_folder}")

                            for third_level_folder in os.listdir(second_level_path):
                                third_level_path = os.path.join(second_level_path, third_level_folder)
                                if os.path.isdir(third_level_path):
                                    words = third_level_folder.split("-")

                                    # Verifica el tipo de carpeta y procesa los documentos
                                    if "ROI mask images" in words:
                                        roi_output_folder = output_dir / "ROI mask images" / class_folder
                                        cropped_output_folder = output_dir / "cropped images" / class_folder
                                        target_size_roi = (800, 1350)
                                        target_size_cropped = (550, 550)

                                        # Crea las carpetas de salida si no existen
                                        if not roi_output_folder.exists():
                                            roi_output_folder.mkdir(parents=True)
                                        if not cropped_output_folder.exists():
                                            cropped_output_folder.mkdir(parents=True)

                                        documents = sorted(os.listdir(third_level_path))
                                        for idx, document in enumerate(documents):
                                            source_path = os.path.join(third_level_path, document)
                                            if os.path.isfile(source_path):
                                                if idx == 0:  # La carpeta asignada a la primera imagen es "cropped images"
                                                    target_path = cropped_output_folder / f"{class_folder}_{first_level_folder}_{second_level_folder}_{document.replace('.dcm', '.png')}"
                                                    preprocess_image(source_path, target_path, target_size_cropped)
                                                elif idx == 1:  # La carpeta asignada a la primera imagen es "ROI mask images"
                                                    target_path = roi_output_folder / f"{class_folder}_{first_level_folder}_{second_level_folder}_{document.replace('.dcm', '.png')}"
                                                    preprocess_image(source_path, target_path, target_size_roi)
                                    elif "full mammogram images" in words:
                                        output_folder = output_dir / "full mammogram images" / class_folder
                                        target_size = (800, 1350)

                                        if not output_folder.exists():
                                            output_folder.mkdir(parents=True)

                                        for document in os.listdir(third_level_path):
                                            source_path = os.path.join(third_level_path, document)
                                            if os.path.isfile(source_path):
                                                target_path = output_folder / f"{class_folder}_{first_level_folder}_{second_level_folder}_{document.replace('.dcm', '.png')}"
                                                preprocess_image(source_path, target_path, target_size)
                                    elif "cropped images" in words:
                                        output_folder = output_dir / "cropped images" / class_folder
                                        target_size = (550, 550)

                                        if not output_folder.exists():
                                            output_folder.mkdir(parents=True)

                                        for document in os.listdir(third_level_path):
                                            source_path = os.path.join(third_level_path, document)
                                            if os.path.isfile(source_path):
                                                target_path = output_folder / f"{class_folder}_{first_level_folder}_{second_level_folder}_{document.replace('.dcm', '.png')}"
                                                preprocess_image(source_path, target_path, target_size)

if __name__ == "__main__":
    reorganize_images()


### Carga de directorios e imágenes

#### Carga de directorios 

In [ ]:
mass_train_dir = r'C:\Users\mcamp\OneDrive\Documentos\TFGDEF2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Training'
mass_test_dir = r'C:\Users\mcamp\OneDrive\Documentos\TFGDEF2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Test'

#### Carga de imágenes

##### Variables globales

In [ ]:
target_size_full = (800, 1350)
target_size_cropped = (550, 550)
batch_size = 16

##### Procesamiento de imágenes

In [ ]:
def load_images_from_mass_structure(base_dir, image_type, target_size):
    """
    Función destinada a la carga de imágenes PNG desde los directorios donde se encuentran los datasets de entrenamiento
    y validación.
    """
    images = []
    labels = []
    class_dir = os.path.join(base_dir, image_type)
    class_names = sorted(os.listdir(class_dir))

    for label, class_name in enumerate(class_names):
        class_path = os.path.join(class_dir, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if file_name.endswith(".png"):
                    img_path = os.path.join(class_path, file_name)
                    try:
                        with Image.open(img_path) as img:
                            if img.mode != "L":
                                img = img.convert("L") 
                            img = img.resize(target_size, Image.Resampling.LANCZOS)
                            image_array = img_to_array(img)
                            images.append(image_array)
                            labels.append(label)
                        print(f"Imagen {img_path} procesada")
                    except Exception as e:
                        print(f"Error al procesar la imagen {img_path}: {e}")

    return np.array(images), np.array(labels), class_names



##### Carga de imágenes de entrenamiento

In [ ]:
full_train_images, full_train_labels, class_names = load_images_from_mass_structure(
    mass_train_dir, "full mammogram images", target_size=target_size_full
)
cropped_train_images, cropped_train_labels, _ = load_images_from_mass_structure(
    mass_train_dir, "cropped images", target_size=target_size_cropped
)

##### Carga de imágenes de validación

In [ ]:
full_val_images, full_val_labels, _ = load_images_from_mass_structure(
    mass_test_dir, "full mammogram images", target_size=target_size_full
)
cropped_val_images, cropped_val_labels, _ = load_images_from_mass_structure(
    mass_test_dir, "cropped images", target_size=target_size_cropped
)

##### Comprobación y corrección de dimensiones

In [ ]:
# Se verifican las dimensiones iniciales
print("Formas iniciales:")
print("Full train:", full_train_images.shape)
print("Cropped train:", cropped_train_images.shape)
print("Full val:", full_val_images.shape)
print("Cropped val:", cropped_val_images.shape)

# Se corrigen las dimensiones si están invertidas
full_train_images = np.transpose(full_train_images, (0, 2, 1, 3))
cropped_train_images = np.transpose(cropped_train_images, (0, 2, 1, 3))
full_val_images = np.transpose(full_val_images, (0, 2, 1, 3))
cropped_val_images = np.transpose(cropped_val_images, (0, 2, 1, 3))

# Se verifican las dimensiones después de la corrección
print("Formas corregidas:")
print("Full train:", full_train_images.shape)
print("Cropped train:", cropped_train_images.shape)
print("Full val:", full_val_images.shape)
print("Cropped val:", cropped_val_images.shape)

## Modelo inicial

### Creación del modelo

In [ ]:
def create_multimodal_model(input_shape_full, input_shape_cropped, num_classes):
    """
    Función destinada a la creacción del modelo multimodal empleado en este TFG.

    La arquitectura se compone de dos submodelos independientes, uno para las mamografías completas y otro para las 
    recortadas donde se muestra la localización ampliada del tumor. Ambos submodelos siguen una estructura similar de
    capas convolucionales pero funcionan como flujos de procesamiento separados hasta que sus salidas son combinadas
    con el fin de optimizar la eficiencia del entrenamiento y reducir la sobrecarga computacional.
    """
    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu")(concatenated)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="class_output")(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_multimodal = create_multimodal_model(
    input_shape_full=(800, 1350, 1), 
    input_shape_cropped=(550, 550, 1), 
    num_classes=len(class_names)
)

### Entrenamiento del modelo

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación
history = model_multimodal.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

### Graficación de los resultados

In [ ]:
def plot_loss_acc(history):
    """
    Función destinada a la generación de dos gráficas que muestran la evolución del entrenamiento y validación de un
    modelo de aprendizaje automático a lo largo de las épocas extrayendo la información del historial de información.
    """
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle('Gráficas de la precisión y la pérdida del entrenamiento y la validación')
    for i, (data, label) in enumerate(zip([(acc, val_acc), (loss, val_loss)], ["Precisión", "Pérdida"])):
        ax[i].plot(epochs, data[0], 'r', label="Entrenamiento " + label)
        ax[i].plot(epochs, data[1], 'b', label="Validación " + label)
        ax[i].legend()
        ax[i].set_xlabel('epochs')
    plt.show()

In [ ]:
plot_loss_acc(history)

## Optimización del modelo inicial a partir de la implementación de diversas técnicas

### Modelo con Aumento de Datos añadido

#### Especificación de los parámetros empleados en el Aumento de Datos

In [ ]:
def create_data_augmentation():
    """
    Función destinada a la especificación de los parámetros empleados en el Aumento de Datos.
    """
    return tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.3),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(0.2, 0.2)
    ])

#### Definición del nuevo modelo

In [ ]:
def create_multimodal_model_with_augmentation(input_shape_full, input_shape_cropped, num_classes):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Aumento de Datos
    a partir del modelo multimodal anterior.
    """
    data_augmentation = create_data_augmentation()

    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = data_augmentation(input_full)  # Se aplica el Aumento De Datos
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(x_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = data_augmentation(input_cropped)  # Se aplica el Aumento De Datos
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu")(concatenated)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="class_output")(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_with_augmentation = create_multimodal_model_with_augmentation(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names)
)

#### Entrenamiento del nuevo modelo

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación
history_with_augmentation = model_with_augmentation.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_augmentation)

### Modelo con Abandono añadido

#### Definición del nuevo modelo

In [ ]:
def create_multimodal_model_with_dropout(input_shape_full, input_shape_cropped, num_classes):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Abandono
    a partir del modelo multimodal anterior sin Aumento de Datos incluido.
    """
    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Dropout
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu")(concatenated)
    x = tf.keras.layers.Dropout(0.2)(x)  # Dropout
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="class_output")(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def create_multimodal_model_with_dropout_and_aug(input_shape_full, input_shape_cropped, num_classes):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Abandono
    a partir del modelo multimodal anterior con Aumento de Datos incluido.
    """
    data_augmentation = create_data_augmentation()

    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = data_augmentation(input_full)
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = data_augmentation(input_cropped)
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Dropout
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu")(concatenated)
    x = tf.keras.layers.Dropout(0.2)(x)  # Dropout
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="class_output")(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_with_dropout = create_multimodal_model_with_dropout(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names)
)

In [ ]:
model_with_dropout_and_aug = create_multimodal_model_with_dropout_and_aug(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names)
)

#### Entrenamiento del nuevo modelo

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación sin Aumento de Datos incluido
history_with_dropout = model_with_dropout.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación con Aumento de Datos incluido
history_with_dropout_and_aug = model_with_dropout_and_aug.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_dropout)

In [ ]:
plot_loss_acc(history_with_dropout_and_aug)

### Modelo con Normalización por Lotes añadido

#### Definición del nuevo modelo

In [ ]:
def create_multimodal_model_with_bn(input_shape_full, input_shape_cropped, num_classes):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Normalización
    por Lotes a partir del modelo multimodal anterior sin Aumento de Datos incluido.
    """
    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Batch Normalization
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu")(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="class_output")(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def create_multimodal_model_with_bn_and_aug(input_shape_full, input_shape_cropped, num_classes):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Normalización
    por Lotes a partir del modelo multimodal anterior con Aumento de Datos incluido.
    """
    data_augmentation = create_data_augmentation()

    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = data_augmentation(input_full)
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = data_augmentation(input_cropped)
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Batch Normalization
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu")(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", name="class_output")(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_with_bn = create_multimodal_model_with_bn(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names)
)

In [ ]:
model_with_bn_and_aug = create_multimodal_model_with_bn_and_aug(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names)
)

#### Entrenamiento del nuevo modelo

In [ ]:
history_with_bn = model_with_bn.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

In [ ]:
history_with_bn_and_aug = model_with_bn_and_aug.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_bn)

In [ ]:
plot_loss_acc(history_with_bn_and_aug)

### Modelo con Regularización L1 añadido

#### Definición del nuevo modelo

In [ ]:
def create_multimodal_model_with_l1(input_shape_full, input_shape_cropped, num_classes, l1_value):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Regularización L1
    a partir del modelo multimodal anterior sin Aumento de Datos incluido.
    """
    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Regularización L1
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L1(l1=l1_value))(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=L1(l1=l1_value))(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def create_multimodal_model_with_l1_and_aug(input_shape_full, input_shape_cropped, num_classes, l1_value):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Regularización L1
    a partir del modelo multimodal anterior con Aumento de Datos incluido.
    """
    data_augmentation = create_data_augmentation()

    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = data_augmentation(input_full)
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = data_augmentation(input_cropped)
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Regularización L1
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L1(l1=l1_value))(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=L1(l1=l1_value))(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_with_l1 = create_multimodal_model_with_l1(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names),
    l1_value=0.01
)

In [ ]:
model_with_l1_and_aug = create_multimodal_model_with_l1_and_aug(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names),
    l1_value=0.01
)

#### Entrenamiento del nuevo modelo

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación sin Aumento de Datos incluido
history_with_l1 = model_with_l1.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación con Aumento de Datos incluido
history_with_l1_and_aug = model_with_l1_and_aug.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_l1)

In [ ]:
plot_loss_acc(history_with_l1_and_aug)

### Modelo con Regularización L2 añadido

#### Definición del nuevo modelo

In [ ]:
def create_multimodal_model_with_l2(input_shape_full, input_shape_cropped, num_classes, l2_value):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Regularización L2
    a partir del modelo multimodal anterior sin Aumento de Datos incluido.
    """
    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Regularización L2
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L2(l2=l2_value))(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=L2(l2=l2_value))(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def create_multimodal_model_with_l2_and_aug(input_shape_full, input_shape_cropped, num_classes, l2_value):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye la técnica denominada como Regularización L2
    a partir del modelo multimodal anterior con Aumento de Datos incluido.
    """
    data_augmentation = create_data_augmentation()

    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = data_augmentation(input_full)
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = data_augmentation(input_cropped)
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Regularización L2
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L2(l2=l2_value))(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=L2(l2=l2_value))(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_with_l2 = create_multimodal_model_with_l2(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names),
    l2_value=0.01
)

In [ ]:
model_with_l2_and_aug = create_multimodal_model_with_l2_and_aug(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names),
    l2_value=0.01
)

#### Entrenamiento del nuevo modelo

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación sin Aumento de Datos incluido
history_with_l2 = model_with_l2.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación con Aumento de Datos incluido
history_with_l2_and_aug = model_with_l2_and_aug.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_l2)

In [ ]:
plot_loss_acc(history_with_l2_and_aug)

### Modelo con Regularización L1 y L2 añadido

#### Definición del nuevo modelo

In [ ]:
def create_multimodal_model_with_l1l2(input_shape_full, input_shape_cropped, num_classes, l1_value, l2_value):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye las técnicas denominadas como Regularización 
    L1 y L2 a partir del modelo multimodal anterior sin Aumento de Datos incluido.
    """
    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Regularización L1 y L2
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L1L2(l1=l1_value, l2=l2_value))(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=L1L2(l1=l1_value, l2=l2_value))(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def create_multimodal_model_with_l1l2_and_aug(input_shape_full, input_shape_cropped, num_classes, l1_value, l2_value):
    """
    Función destinada a la creacción de un nuevo modelo multimodal que incluye las técnicas denominadas como Regularización 
    L1 y L2 a partir del modelo multimodal anterior con Aumento de Datos incluido.
    """
    data_augmentation = create_data_augmentation()

    # Submodelo para imágenes completas
    input_full = tf.keras.Input(shape=input_shape_full, name="full_mammogram_input")
    x_full = data_augmentation(input_full)
    x_full = tf.keras.layers.Rescaling(1.0 / 255)(input_full)
    x_full = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_full)
    x_full = tf.keras.layers.BatchNormalization()(x_full)
    x_full = tf.keras.layers.MaxPooling2D((2, 2))(x_full)
    x_full = tf.keras.layers.Flatten()(x_full)

    # Submodelo para imágenes recortadas
    input_cropped = tf.keras.Input(shape=input_shape_cropped, name="cropped_image_input")
    x_cropped = data_augmentation(input_cropped)
    x_cropped = tf.keras.layers.Rescaling(1.0 / 255)(input_cropped)
    x_cropped = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x_cropped)
    x_cropped = tf.keras.layers.BatchNormalization()(x_cropped)
    x_cropped = tf.keras.layers.MaxPooling2D((2, 2))(x_cropped)
    x_cropped = tf.keras.layers.Flatten()(x_cropped)

    # Concatenación y capas finales con Regularización L1 y L2
    concatenated = tf.keras.layers.Concatenate()([x_full, x_cropped])
    x = tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=L1L2(l1=l1_value, l2=l2_value))(concatenated)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=L1L2(l1=l1_value, l2=l2_value))(x)

    model = tf.keras.Model(inputs=[input_full, input_cropped], outputs=output)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model_with_l1l2 = create_multimodal_model_with_l1l2(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names),
    l1_value=0.01,
    l2_value=0.01
)

In [ ]:
model_with_l1l2_and_aug = create_multimodal_model_with_l1l2_and_aug(
    input_shape_full=(800, 1350, 1),
    input_shape_cropped=(550, 550, 1),
    num_classes=len(class_names),
    l1_value=0.01,
    l2_value=0.01
)

#### Entrenamiento del nuevo modelo

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación sin Aumento de Datos incluido
history_with_l1l2 = model_with_l1l2.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación com Aumento de Datos incluido
history_with_l1l2_and_aug = model_with_l1l2_and_aug.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_l1l2)

In [ ]:
plot_loss_acc(history_with_l1l2_and_aug)

### Modelo con Detección Temprana añadida

#### Especificación de la clase que define la Detección Temprana

In [ ]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    """
    Esta clase define una callback personalizada de Detección Temprana para Keras. 
    Su función es detener automáticamente el entrenamiento del modelo si se cumplen las condiciones determinadas en la
    función on_epoch_end.
    """
    def on_epoch_end(self, epoch, logs=None):
        """
        Función destinada a la detección automática del entrenamiento del modelo si se cumplen simultáneamente las
        siguientes condiciones:
          - La excatitud del entrenamiento es mayor o igual a 0.95.
          - La exactitud de la validación es mayor o igual a 0.90.
        """
        if logs["accuracy"] >= 0.95 and logs["val_accuracy"] >= 0.90:
            self.model.stop_training = True
            print("\nSe alcanzaron los criterios de Detección Temprana.")

#### Entrenamiento con Detección Temprana añadida

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación sin Aumento de Datos incluido
history_with_early_stopping = model_with_l1l2.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12,
    callbacks=[EarlyStoppingCallback()]
)

In [ ]:
# Se almacena toda la información del proceso de entrenamiento y validación con Aumento de Datos incluido
history_with_early_stopping_and_aug = model_with_l1l2_and_aug.fit(
    {"full_mammogram_input": full_train_images, "cropped_image_input": cropped_train_images},
    full_train_labels,
    validation_data=(
        {"full_mammogram_input": full_val_images, "cropped_image_input": cropped_val_images},
        full_val_labels
    ),
    epochs=10,
    batch_size=12,
    callbacks=[EarlyStoppingCallback()]
)

#### Graficación de los resultados

In [ ]:
plot_loss_acc(history_with_early_stopping)

In [ ]:
plot_loss_acc(history_with_early_stopping_and_aug)

## YOLO

## Preprocesamiento del dataset

### Reorganización del dataset

In [ ]:
# Se determinan las rutas de los archivos
base_dir = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P3\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Training"
output_dir = Path(r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\train")

def create_main_folders():
    """
    Función destinada a la confirmación de la existencia de la carpeta de destino.
    """
    output_dir.mkdir(parents=True, exist_ok=True)

def process_image_no_resize(source_path, target_path):
    """
    Función destinada al procesamiento y guardado de la imagen sin recorte ni resize.
    Lee un archivo DICOM y lo guarda como .png sin redimensionar.
    """
    try:
        dicom_data = pydicom.dcmread(source_path)
        image_array = dicom_data.pixel_array

        # Convierte a 8 bits si fuera necesario
        if image_array.max() > 255:
            image_array = (image_array / image_array.max() * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)

        # Guarda la imagen en su tamaño original
        cv2.imwrite(str(target_path), image_array)
        print(f"Guardada sin redimensionar en: {target_path}")

    except Exception as e:
        print(f"Error al procesar {source_path}: {e}")

def reorganize_images():
    """
    Función principal destinada a la reorganización de imágenes con la misma estructura sin redimensionar.
    """
    create_main_folders()

    for class_folder in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_folder)
        if os.path.isdir(class_path):  # Verifica si es un directorio (clase)
            print(f"Clase: {class_folder}")

            for first_level_folder in os.listdir(class_path):
                first_level_path = os.path.join(class_path, first_level_folder)
                if os.path.isdir(first_level_path):  # Verifica si es un directorio
                    print(f"Primer nivel: {first_level_folder}")

                    for second_level_folder in os.listdir(first_level_path):
                        second_level_path = os.path.join(first_level_path, second_level_folder)
                        if os.path.isdir(second_level_path):  # Verifica si es un directorio
                            print(f"Segundo nivel: {second_level_folder}")

                            for third_level_folder in os.listdir(second_level_path):
                                third_level_path = os.path.join(second_level_path, third_level_folder)
                                if os.path.isdir(third_level_path):  # Verifica si es un directorio
                                    words = third_level_folder.split("-")

                                    # full mammogram images
                                    if "full mammogram images" in words:
                                        # Procesar las imágenes "full mammogram images"
                                        for document in os.listdir(third_level_path):
                                            source_path = os.path.join(third_level_path, document)
                                            if os.path.isfile(source_path):
                                                base_name = f"{first_level_folder}_{second_level_folder}_{third_level_folder}"
                                                target_path = output_dir / f"{base_name}.png"
                                                process_image_no_resize(source_path, target_path)

                                    # ROI mask images
                                    elif "ROI mask images" in words:
                                        # Carpeta de salida para ROI en PNG (sin subcarpeta de clase)
                                        roi_output_folder = output_dir / "ROI mask images"

                                        # Crear las carpetas si no existen
                                        roi_output_folder.mkdir(parents=True, exist_ok=True)

                                        documents = sorted(os.listdir(third_level_path))  # Archivos .dcm, etc.
                                        for idx, document in enumerate(documents):
                                            source_path = os.path.join(third_level_path, document)
                                            if os.path.isfile(source_path):
                                                base_name = f"{first_level_folder}_{second_level_folder}_{third_level_folder}_{idx}"
                                                if idx == 1:
                                                    # Guardar la segunda imagen en "ROI mask images"
                                                    roi_target_path = roi_output_folder / f"{base_name}.png"
                                                    process_image_no_resize(source_path, roi_target_path)
                                                else: pass
                                    else: pass

if __name__ == "__main__":
    reorganize_images()


In [ ]:
def generate_yolo_label_from_roi_png(roi_png_path, label_path, class_id=0):
    """
    Función destinada a la extracción del bounding box de la ROI mask y generación del .txt en formato YOLO dada una imagen.
    Procesa una máscara ROI en formato PNG para generar etiquetas YOLO.
    Asume que la máscara tiene fondo negro (0) y la ROI en blanco (>127).
    """
    try:
        mask_array = cv2.imread(str(roi_png_path), cv2.IMREAD_GRAYSCALE)

        if mask_array is None:
            print(f"No se pudo leer la imagen: {roi_png_path}")
            return

        height, width = mask_array.shape

        # Binariza
        _, thresh = cv2.threshold(mask_array, 127, 255, cv2.THRESH_BINARY)

        # Realiza operaciones morfológicas
        kernel = np.ones((5, 5), np.uint8)
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = [c for c in contours if cv2.contourArea(c) > 50]

        if not contours:
            print(f"No se encontraron contornos en {roi_png_path.name}")
            return

        # Guarda la etiqueta YOLO
        with open(label_path, "w") as f:
            for c in contours:
                x, y, w_box, h_box = cv2.boundingRect(c)

                x_center = (x + w_box / 2) / width
                y_center = (y + h_box / 2) / height
                w_norm = w_box / width
                h_norm = h_box / height

                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        print(f"Etiqueta YOLO generada: {label_path}")

    except Exception as e:
        print(f"Error al procesar {roi_png_path.name}: {e}")


def process_roi_png_folder_in_place(roi_folder_path, class_id=0):
    """
    Función destinada a la extracción del bounding box de la ROI mask y generación del .txt en formato YOLO dada una carpeta.
    Recorre las imágenes .png de una carpeta procesando sus máscaras ROI para generar etiquetas YOLO.
    """
    roi_folder = Path(roi_folder_path)
    label_folder = roi_folder / "labels"
    label_folder.mkdir(exist_ok=True)

    for roi_png in sorted(roi_folder.glob("*.png")):
        base_name = roi_png.stem
        label_output_path = label_folder / f"{base_name}.txt"

        print(f"Procesando ROI PNG: {roi_png.name}")
        generate_yolo_label_from_roi_png(roi_png, label_output_path, class_id=class_id)


In [ ]:
roi_folder_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Test\ROI mask images\TODO"
process_roi_png_folder_in_place(roi_folder_path, class_id=0)

### Agrupación de los archivos .txt que hacen referencia a la misma mama

In [ ]:
def grouping_and_renaming_labels(carpeta_labels, eliminar_originales=True):
    """ 
    Función destinada, dada una carpeta con archivos .txt, a la agrupación en función de los nombres de los archivos 
    prescindiendo del número final puesto que hacen referencia a la misma mama.
    De esta forma, realiza una de las siguientes acciones para cada grupo:
    - Si hay más de un archivo en el grupo, los combina en un solo archivo de salida.
    - Si solo hay un archivo en el grupo, lo copia sin cambios a un archivo de salida.
    Los archivos originales se eliminan después de ser procesados, si el parámetro 'eliminar_originales' es True.
    La función imprime mensajes sobre el progreso de cada acción: combinación, copia o eliminación.
    """
    carpeta = Path(carpeta_labels)
    archivos = sorted([f for f in carpeta.glob("*.txt") if f.is_file()])

    grupos = defaultdict(list)

    for archivo in archivos:
        nombre = archivo.stem  # sin .txt
        partes = nombre.split("_")
        if partes[-1].isdigit():
            clave = "_".join(partes[:-1])
            grupos[clave].append(archivo)

    for clave, lista_archivos in grupos.items():
        archivo_salida = carpeta / f"{clave}.txt"

        if len(lista_archivos) > 1:
            with open(archivo_salida, "w") as salida:
                for archivo in sorted(lista_archivos):
                    with open(archivo, "r") as f:
                        salida.writelines(f.readlines())
            print(f"Combinado: {archivo_salida.name} ({len(lista_archivos)} archivos)")

        elif len(lista_archivos) == 1:
            shutil.copy(lista_archivos[0], archivo_salida)
            print(f"Copiado: {lista_archivos[0].name} ➜ {archivo_salida.name}")

        if eliminar_originales:
            for archivo in lista_archivos:
                archivo.unlink()
                print(f"Eliminado: {archivo.name}")

In [ ]:
label_folder_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Test\ROI mask images\TODO\labels"
grouping_and_renaming_labels(label_folder_path, eliminar_originales=True)

### Confirmación de que cada imagen png posea un archivo txt con su nombre

In [ ]:
def compare_names_between_pngs_and_txts(folder1, folder2):
    """
    Función destinada a la comparación de los nombres de archivos (sin extensión) entre dos carpetas.
    Imprime si son iguales o muestra las diferencias.
    """
    folder1 = Path(folder1)
    folder2 = Path(folder2)

    files1 = {f.stem for f in folder1.glob("*") if f.is_file()}
    files2 = {f.stem for f in folder2.glob("*") if f.is_file()}

    only_in_1 = files1 - files2
    only_in_2 = files2 - files1

    if not only_in_1 and not only_in_2:
        print("Todos los archivos coinciden entre ambas carpetas.")
        return True
    else:
        print("Hay diferencias entre las carpetas:")
        if only_in_1:
            print(f"Solo en {folder1.name}:")
            for f in sorted(only_in_1):
                print(f"  - {f}")
        if only_in_2:
            print(f"Solo en {folder2.name}:")
            for f in sorted(only_in_2):
                print(f"  - {f}")
        return False


In [ ]:
compare_names_between_pngs_and_txts(
    r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Training\ROI mask images\TODO(UNIDO)\labels",
    r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\Mass_Training\full mammogram images\TODO"
)

## Creacción y entrenamiento de los modelos

### Modelo sin Optimizador AdamW ni flip horizontal

In [ ]:
def yolo():
    """
    Función destinada al entrenamiento de un modelo YOLOv8 con el fin de localizar con la mayor exactitud posible 
    la localización de los tumores dado el dataset de mamografías.
    """
    # Carga el modelo base "yolov8n.pt"
    model = YOLO("yolov8n.pt")

    # Entrena durante 100 épocas con imágenes de tamaño 640x640 y un batch de 8
    model.train(
        data=r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\mammo.yaml",    
        epochs=100,
        imgsz=640,
        batch=8,
        name="mammo_yolo",  # Guarda el experimento bajo el nombre "mammo_yolo"
        plots=True          # Genera gráficas del entrenamiento.
    )

In [ ]:
yolo()

### Modelo con Optimizador AdamW y flip horizontal

In [ ]:
def yolo_adamw_flip():
    """
    Función destinada al entrenamiento de un modelo YOLOv8 incluyendo el uso del optimizador AdamW y la técnica denominada
    como Aumento de Datos con el fin de localizar con la mayor exactitud posible la localización de los tumores dado
    el dataset de mamografías.
    """
    # Carga el modelo base "yolov8n.pt" y lo entrena usando la configuración definida en el archivo YAML
    model = YOLO("yolov8n.pt")

    # Entrena durante 100 épocas con imágenes de tamaño 640x640 y un batch de 8
    model.train(
        data=r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\mammo.yaml",    
        epochs=100,
        imgsz=640,
        batch=8,
        name="mammo_yolo_adamw_flip",
        augment=True,    # Habilita el augmentation
        fliplr=0.5,      # Probabilidad de flip horizontal = 50%
        flipud=0.0,      # Probabilidad de flip vertical = 0% (desactivado)
        optimizer="AdamW",
        plots=True 
    )

In [ ]:
yolo_adamw_flip()

## Evaluación gráfica del rendimiento de ambos modelos durante el entrenamiento y la validación

In [ ]:
%matplotlib inline

results_png_path = os.path.join("runs\detect\mammo_yolo_adamw_flip7", "results.png")
img = plt.imread(results_png_path)
plt.figure(figsize=(15,15))
plt.imshow(img)
plt.axis('off')
plt.show()


In [ ]:
%matplotlib inline

results_png_path = os.path.join("runs\detect\mammo_yolo_flip", "results.png")
img = plt.imread(results_png_path)
plt.figure(figsize=(15,15))
plt.imshow(img)
plt.axis('off')
plt.show()

## Comparación de datos

### Visualización de las etiquetas originales

In [ ]:
def draw_box_yolo_style(
    img, 
    x1, y1, x2, y2, 
    label="tumor", 
    score=0.0, 
    font_scale=0.8, 
    font_thickness=2,
    color=(0, 0, 255),
    min_box_size=50
):
    """
    Función destinada a la creacción sobre la imagen de una caja basándose en el estilo de YOLO, es decir, un rectángulo rojo 
    señalando la zona afectada con un pequeño recuadro encima de él que continene la etiqueta que indica la puntuación sobre 
    cuánta exactitud posee la posición del rectángulo predicho por el modelo respecto a la posición real de éste.
    """

    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

    # Dibuja el rectángulo principal
    box_w = x2 - x1
    box_h = y2 - y1
    if box_w < min_box_size or box_h < min_box_size:
        thickness = 3
    else:
        thickness = 2

    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness, cv2.LINE_AA)

    # Crea el texto calculando además su tamaño
    text = f"{label} {score:.2f}"
    font = cv2.FONT_HERSHEY_SIMPLEX

    (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, font_thickness)

    # Posiciona la etiqueta
    text_x = x1
    text_y = y1 - 5

    img_h, img_w = img.shape[:2]
    top_rect_y = text_y - text_h - baseline
    if top_rect_y < 0:
        text_y = y1 + text_h + baseline + 5
        top_rect_y = y1

    # Dibuja el recuadro relleno para el texto
    cv2.rectangle(
        img,
        (text_x, top_rect_y),
        (text_x + text_w, text_y),
        color,
        -1
    )

    # Posiciona el texto en blanco encima
    cv2.putText(
        img,
        text,
        (text_x, text_y - baseline),
        font,
        font_scale,
        (255, 255, 255),
        font_thickness,
        cv2.LINE_AA
    )


In [ ]:
def draw_yolo_labels(image_path, label_path):
    """
    Función destinada a la visualización de las mamografías destacando la localización de los tumores existentes  
    simulando lo mejor posible el estilo propio del output de YOLO. 
    """
    # Lee la imagen .png
    img = cv2.imread(image_path)
    if img is None:
        print("No se pudo cargar la imagen:", image_path)
        return

    h, w = img.shape[:2]

    # Lee el archivo .txt con las etiquetas YOLO asociado a la imagen tomada anteriormente
    try:
        with open(label_path, "r") as f:
            lines = f.readlines()
    except:
        print("No se pudo leer el archivo de etiquetas:", label_path)
        return

    # Obtiene la información procedente de las etiquetas cuyo formato es: (class_id x_center y_center w_norm h_norm)
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        class_id, x_center_norm, y_center_norm, w_norm, h_norm = parts
        x_center_norm = float(x_center_norm)
        y_center_norm = float(y_center_norm)
        w_box_norm = float(w_norm)
        h_box_norm = float(h_norm)

        # Convierte a píxeles reales
        x_center = x_center_norm * w
        y_center = y_center_norm * h
        w_box = w_box_norm * w
        h_box = h_box_norm * h

        x1 = x_center - (w_box / 2)
        y1 = y_center - (h_box / 2)
        x2 = x_center + (w_box / 2)
        y2 = y_center + (h_box / 2)

        # Dibuja los rectángulos propios eel estilo YOLO
        draw_box_yolo_style(
            img, 
            x1, y1, x2, y2, 
            label="tumor", 
            score=1.0,
            font_scale=3,
            font_thickness=14
        )

    # Visualiza los resultados
    import matplotlib.pyplot as plt
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6,6))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()


### Visualización de las etiquetas predichas por del modelo

In [ ]:
def yolo_predict(image_path):
    """
    Función destinada a la visualización de las mamografías destacando la localización de los tumores existentes 
    según las predicciones obtenidas por YOLO.
    """
    # Carga el modelo entrenado
    model = YOLO("runs/detect/mammo_yolo2/weights/best.pt")

    # Predecice la posición de los tumores con YOLO
    results = model.predict(source=image_path)

    # Obtiene el resultado
    res = results[0]

    # Dibujan las cajas en la imagen
    img_annotated = res.plot()

    # Muestra el resultado
    plt.imshow(img_annotated)
    plt.axis('off')
    plt.show()

### Comparación gráfica entre las etiquetas reales y las predichas por el modelo YOLO

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\train\Mass_Training_P_00039_RIGHT_CC.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\train\Mass_Training_P_00039_RIGHT_CC.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\train\Mass_Training_P_00914_LEFT_CC.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\train\Mass_Training_P_00914_LEFT_CC.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\train\Mass_Training_P_00148_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\train\Mass_Training_P_00148_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\train\Mass_Training_P_01529_RIGHT_CC.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\train\Mass_Training_P_01529_RIGHT_CC.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\train\Mass_Training_P_01083_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\train\Mass_Training_P_01083_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00200_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00200_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00677_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00677_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00773_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00773_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_01800_LEFT_CC.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_01800_LEFT_CC.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00662_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00662_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00629_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00629_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00099_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00099_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00126_RIGHT_CC.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00126_RIGHT_CC.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00758_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00758_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_01912_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_01912_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_01578_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_01578_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_01323_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_01323_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_01251_LEFT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_01251_LEFT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_01235_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_01235_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00591_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00591_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)

In [ ]:
%matplotlib inline

image_path = r"C:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\images\val\Mass_Test_P_00391_RIGHT_MLO.png"
label_path = r"c:\Users\mcamp\OneDrive\Documentos\TFG_P2\manifest-ZkhPvrLo5216730872708713142\CBIS-DDSM\YOLO\labels\val\Mass_Test_P_00391_RIGHT_MLO.txt"
draw_yolo_labels(image_path, label_path)
yolo_predict(image_path)